In [1]:
#import necessary tools
import re
from string import digits
import pandas as pd
import numpy as np

In [2]:
#read the documents
with open('data/213LB_b2.txt','rb') as f:
    contents = f.read()
    contents = contents.decode("utf-16").rstrip("\n")
    contents = contents.split("\r\n") #maybe need to strip \t?

In [3]:
Procedure=list()
img =[]
Choice_RESP = []
Choice_RT=[]


#loop through each line
for line in range(len(contents)-15):
    
    #this line of "running: listlike/black" is in the fixed position of each of 4 conditions (b1b2l1l2)
    #therefore, we use it as our anchor to locate meaningful blocks

    black_check = contents[line+5].startswith("\tRunning: ListDark") == True
    
    if contents[line] == '\t*** LogFrame Start ***' and black_check:
        
        #loop inside the meaningful block                
        for inner_line in range(15):
            
            new_line = line + inner_line
            
            #mark image
            if contents[new_line].startswith("\timg: "):
                imga = contents[new_line].split("img: ",1)[1] 
                #print("imag is", imga)
                img.append(imga)
                
             
            #mark choice Like response 
            if contents[new_line].startswith("\tSlideD1.RESP: ") and contents[new_line].split("\tSlideD1.RESP: ",1) != ['', '']:

                Choice_RESPa = contents[new_line].split("\tSlideD1.RESP: ",1)[1]
                Choice_RESPa =[''.join(c for c in Choice_RESPa if c in digits)]
                Choice_RESP=Choice_RESP+Choice_RESPa
                
            #mark choice Like response time
            if contents[new_line].startswith("\tSlideD1.RT: ") and contents[new_line].split("\tSlideD1.RT: ",1) != ['']:
                
                Choice_RTa = contents[new_line].split("\tSlideD1.RT: ",1)[1]
                #print("rt is", Choice_RTa)
                Choice_RTa =[''.join(c for c in Choice_RTa if c in digits)]
                Choice_RTa = list([float(i) for i in Choice_RTa])
                Choice_RT=Choice_RT+Choice_RTa


#Assign variable names  
        
img = [int(i) for i in img] 
img_df = pd.DataFrame(img)
img_df.columns = ['imgB']

Choice_RESP = [int(i) for i in Choice_RESP] 
Choice_RESP_df = pd.DataFrame(Choice_RESP)
Choice_RESP_df.columns = ['rateB']

Choice_RT = [int(i) for i in Choice_RT] 
Choice_RT_df = pd.DataFrame(Choice_RT)
Choice_RT_df.columns = ['rtB']


In [4]:
#merge the data
sub_id = contents[16].split("Subject: ",1)[1] 
sub_df = pd.DataFrame(np.repeat(sub_id, img_df.shape[0], axis = 0))
sub_df.columns = ['sub']


all = pd.concat([sub_df.reset_index(drop=True), img_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Choice_RESP_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Choice_RT_df], axis=1)

all

,sub,imgB,rateB,rtB
0,213,1,4,8368
1,213,2,3,1899
2,213,3,4,2955
3,213,4,6,2702
4,213,5,5,5435
...,...,...,...,...
115,213,116,3,3864
116,213,117,6,6713
117,213,118,4,3699
118,213,119,7,5610


In [5]:
#remove last NA line if needed

#all_all= all[:-1]

all_all= all


all_all

,sub,imgB,rateB,rtB
0,213,1,4,8368
1,213,2,3,1899
2,213,3,4,2955
3,213,4,6,2702
4,213,5,5,5435
...,...,...,...,...
115,213,116,3,3864
116,213,117,6,6713
117,213,118,4,3699
118,213,119,7,5610


In [6]:

#export
all_all.to_excel("rateB_213.xlsx")

#has wrong first column, needs to manually remove